# Scientific Computing: Diffusion Limited Aggregation and Reaction Diffusion

## Imports

In [3]:
import numpy as np
from IPython.display import HTML

import src.solutions as solutions
import src.visualizations as visualizations

In [4]:
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

In [5]:
def visualize_object_grid(obj_grid, stage):
    """
    Visualizes a 2x2 grid of object configurations using binary occupancy maps.

    Parameters:
        obj_grids (list of list of ndarray): A list containing four object grid configurations (2D).
        sizes (list of str): A list of four labels corresponding to the object configurations.
    """
    plt.plot(figsize=(3.1, 3.8))

    # grey for spaces not occupied and black for occupied spaces
    cmap = mcolors.ListedColormap(["lightgrey", "black"])

    plt.imshow(obj_grid, cmap=cmap)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)

    plt.title(f"Object Grids (stage = {stage})")
    # plt.savefig("plots/object_layout.png", dpi=300)
    plt.show()

In [68]:
from scipy.ndimage import binary_dilation
def place_objects(N, size_object=1):
    """
    Randomly places square objects on an NxN grid.

    Parameters:
        N (int): Grid size (N × N).
        num_object (int): Number of objects to place.
        seed (int, optional): Random seed for reproducibility (default=31).
        size_object (int, optional): Side length of each square object (default=4).

    Returns:
        numpy.ndarray: NxN grid with placed objects, where occupied cells are marked as 1.
    """

    object_grid = np.zeros((N, N))
    x = int(N/2 - size_object/2)

    points = [
        (x + j, N -1- k) for j in range(size_object) for k in range(size_object)
    ]

    # set value of indexes that object occupies 1
    object_grid[tuple(zip(*points))] = 1
    

    return object_grid

def generate_stencil(object_grid):
    stencil = np.copy(object_grid)
    # Define a strict Neumann (4-neighbor) structuring element
    neighborhood = np.array([[0, 1, 0],  # Only North, South, West, East
                    [1, 0, 1],
                    [0, 1, 0]])

    # Apply binary dilation to get the stencil
    stencil = binary_dilation(object_grid, structure=neighborhood).astype(int)

    # Ensure original object points remain unchanged
    stencil[object_grid == 1] = 0  # Exclude original object points
    return stencil

In [69]:

def initialize_grid(N, object_grid):
    """
    Generates a grid with the specified dimensions and initializes the boundaries.
    Parameters:
        N (int): Grid size.
        object_grid (np.array): 2D matrix with 1s one places where the object is placed (0s everywhere else)
    """
    assert N > 1, "Grid size must be bigger than 1x1"
    assert object_grid.shape == (N, N), f"object_grid must have the same dimensions as {N, N}"

    grid = np.zeros((N, N))

    grid[object_grid==1] = 1
    return grid

def empty_object_places(grid, stenciltje, object_grid, nu):
    assert object_grid.shape == grid.shape, "object_grid must have the same dimensions as diffusion grid"
    
    emptied_grid = np.copy(grid)
    emptied_grid[stenciltje==0] = 0 

    emptied_grid = np.power(emptied_grid, nu)
    total_sum = emptied_grid.sum()
    assert total_sum > 0, "Initialize object, The Advection Diffusion does not work on an empty grid"
    emptied_grid/=total_sum
    return emptied_grid.flatten()

def sequential_SOR(grid,tol, max_iters, omega, object_grid=None):
    """
    Solves using the Successive Over Relaxtion (SOR) iteration method.

    The update equation is:
        c_{i,j}^{k+1} = (omega/4) * (c_{i+1,j}^{k} + c_{i,j+1}^{k} + c_{i,j+1}^{k} + (1 - omega) c_{i,j}^{k})

    Parameters:
        grid (np.array): Grid (2D matrix).
        tol (float): Convergence tolerance.
        max_iters (int): Maximum number of iterations.
        omega (float): Relaxation factor.

    Returns:
        int: Number of iterations required to reach convergence.
        numpy.ndarray: Final grid after iterations.
    """
    N= len(grid)
    assert N > 1, (
        f"bord is {N}x{N}, but needs to be at least 2*2 for this diffusion implementation"
    )

    # grid initialisation
    # c = initialize_grid(N)

    iter = 0
    delta = float("inf")

    # while not converged
    while delta > tol and iter < max_iters:
        delta = 0

        
        # loop over all cells in the grid (except for y = 0, y=N)
        for i in range(1, N-1):
            for j in range(1, N-1):
                if object_grid is not None and object_grid[(i, j)]:
                    c_next = 1
                    continue
                # retrieve all necessary values (also regarding wrap-around)
                south = grid[i - 1, j] #if i > 0 else grid[N-1, j]
                north = grid[i + 1, j] #if i < N - 1 else grid[0, j]
                west = grid[i, j - 1] #if j > 0 else grid[i, N - 1]
                east = grid[i, j + 1] #if j < N - 1 else grid[i, 0]

                # SOR update equation
                c_next = (omega / 4) * (west + east + south + north) + (1 - omega) * grid[
                    i, j
                ]

                # check for convergence
                delta = max(delta, abs(c_next - grid[i, j]))
                grid[i, j] = c_next

        # borders, derivative is 0 at the borders
        grid[N-1, :] = grid[N-2, :]
        grid[0, :] = grid[1, :]
        grid[:, N-1] = grid[:, N-2]
        grid[:, 0] = grid[:, 1]
        grid[object_grid==1] = 1
        iter += 1

    return iter, grid

In [33]:
def plot_simulation_without_animation(grid, N):
    """
    Generates a visualization of the final state of a 2D diffusion simulation.

    Parameters:
        grids (list of numpy.ndarray): 2D concentration grids from the simulation.
        N (int): Grid size (number of spatial points in each dimension).
    """
    fig, ax = plt.subplots(figsize=(7, 7))

    c_plot = ax.pcolormesh(grid, cmap="viridis", edgecolors="k", linewidth=0.5)

    # plt.imshow(c, cmap="viridis", interpolation="nearest", origin="lower")
    plt.colorbar(c_plot, ax=ax, label="Concentration")
    ax.set_xticks(np.arange(N))
    ax.set_yticks(np.arange(N))
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_title("2D Diffusion")
    plt.show()

def save_grid_to_file(grid, filename="data/grid_output.txt"):
    np.savetxt(filename, grid, fmt="%.6f")  # Save with 6 decimal places for precision
    print(f"Grid saved to {filename}")

def load_grid_from_file(filename="data/grid_output.txt"):
    return np.loadtxt(filename)  # Loads the 2D array back

In [63]:
def normalize_concentration(concentration_grid):
    c_min = concentration_grid.min()
    c_max = 1
    
    # Avoid division by zero in case c_max == c_min
    assert c_min <1, "No non-object spots, breaks Advection diffusion, choose a less dense object grid"
    return (concentration_grid - c_min) / (c_max - c_min)



In [77]:
# packed parameters for easy transfer
N = 100
nu = 6
tol = 1e-3
maxiters = 10000
omega = 1.9
grid_indices = np.arange(N*N)
np.random.seed(22)

In [ ]:
object_grid = place_objects(N)
grid =initialize_grid(N, object_grid)
visualize_object_grid(object_grid, 0)
iters, grid = sequential_SOR(grid, tol, maxiters, omega, object_grid)

In [ ]:
normalized_grid = normalize_concentration(grid)
plot_simulation_without_animation(normalized_grid, N)

save_grid_to_file(grid)

In [ ]:
# prev_grid = np.copy(grid)
# read_ingrid = load_grid_from_file()

# assert np.allclose(prev_grid, grid, atol=1e-5), "grids should be the same"

print(grid)

iter_grid = np.copy(grid)
object_grid_iter = np.copy(object_grid)

for i in range(1000):
    iters, iter_grid = sequential_SOR(iter_grid, tol, maxiters, omega, object_grid_iter)
    print("completed sor")

    stencil = generate_stencil(object_grid_iter)
    normalized_grid = normalize_concentration(iter_grid)
    probs = empty_object_places(normalized_grid, stencil, object_grid_iter, nu)
    selected_index = np.random.choice(grid_indices, p=probs)
    new_index = np.unravel_index(selected_index,iter_grid.shape)
    object_grid_iter[new_index] = 1

    if i%200 == 0:
        visualize_object_grid(object_grid_iter, i)
